<a href="https://colab.research.google.com/github/czz1997/EECE571T/blob/master/EECE571T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Setup**

### Code & Dependencies

In [ ]:
!pip install dominate
!pip install piq
!pip install pytorch-fid

# Load the TensorBoard notebook extension
%load_ext tensorboard

!git clone https://github.com/czz1997/EECE571T.git

### Google Drive

In [ ]:
from google.colab import drive
import os
import shutil
drive.mount("/content/drive", force_remount=True)

# **Dataset Preparation**

### Directories & Soft link

In [ ]:
!rm -rf sample_data
!mkdir /content/datasets  # dataset root
!mkdir /content/EECE571T/datasets
!ln -s /content/datasets /content/EECE571T/datasets # soft link

### Extract

In [ ]:
import os
!mkdir /content/datasets/temp
!unzip -q /content/drive/MyDrive/GT-RAIN_train.zip -d /content/datasets/temp/
!unzip -q /content/drive/MyDrive/GT-RAIN_val.zip -d /content/datasets/temp/
!unzip -q /content/drive/MyDrive/GT-RAIN_test.zip -d /content/datasets/temp/

### Reorganize

In [ ]:
import re
sourceImgPath = r"datasets/temp/GT-RAIN_train"
destRainImgPath = r"datasets/GT-RAIN/trainA/"
destCleanImgPath = r"datasets/GT-RAIN/trainB/"
for scenesFolder in os.listdir(sourceImgPath):
    destCleanSceneFolder = os.path.join(destCleanImgPath,scenesFolder)
    if not os.path.exists(destCleanSceneFolder):
        os.makedirs(destCleanSceneFolder)
    destRainSceneFolder = os.path.join(destRainImgPath,scenesFolder)
    if not os.path.exists(destRainSceneFolder):
        os.makedirs(destRainSceneFolder)

    for img in os.listdir(os.path.join(sourceImgPath, scenesFolder)):
        if re.search(r'.*C-\d{3}$', os.path.splitext(img)[0]):
          shutil.move(os.path.join(sourceImgPath, scenesFolder, img), os.path.join(destCleanSceneFolder, img))
        if re.search(r'.*R-\d{3}$', os.path.splitext(img)[0]):
          shutil.move(os.path.join(sourceImgPath, scenesFolder, img), os.path.join(destRainSceneFolder, img))

In [ ]:
sourceImgPath = r"datasets/temp/GT-RAIN_val"
destRainImgPath = r"datasets/GT-RAIN/valA/"
destCleanImgPath = r"datasets/GT-RAIN/valB/"
for scenesFolder in os.listdir(sourceImgPath):
    destCleanSceneFolder = os.path.join(destCleanImgPath,scenesFolder)
    if not os.path.exists(destCleanSceneFolder):
        os.makedirs(destCleanSceneFolder)
    destRainSceneFolder = os.path.join(destRainImgPath,scenesFolder)
    if not os.path.exists(destRainSceneFolder):
        os.makedirs(destRainSceneFolder)

    for img in os.listdir(os.path.join(sourceImgPath, scenesFolder)):
        if re.search(r'.*C-\d{3}$', os.path.splitext(img)[0]):
          shutil.move(os.path.join(sourceImgPath, scenesFolder, img), os.path.join(destCleanSceneFolder, img))
        if re.search(r'.*R-\d{3}$', os.path.splitext(img)[0]):
          shutil.move(os.path.join(sourceImgPath, scenesFolder, img), os.path.join(destRainSceneFolder, img))

In [ ]:
sourceImgPath = r"datasets/temp/GT-RAIN_test"
destRainImgPath = r"datasets/GT-RAIN/testA/"
destCleanImgPath = r"datasets/GT-RAIN/testB/"
for scenesFolder in os.listdir(sourceImgPath):
    destCleanSceneFolder = os.path.join(destCleanImgPath,scenesFolder)
    if not os.path.exists(destCleanSceneFolder):
        os.makedirs(destCleanSceneFolder)
    destRainSceneFolder = os.path.join(destRainImgPath,scenesFolder)
    if not os.path.exists(destRainSceneFolder):
        os.makedirs(destRainSceneFolder)

    for img in os.listdir(os.path.join(sourceImgPath, scenesFolder)):
        if re.search(r'.*C-\d{3}$', os.path.splitext(img)[0]):
          shutil.move(os.path.join(sourceImgPath, scenesFolder, img), os.path.join(destCleanSceneFolder, img))
        if re.search(r'.*R-\d{3}$', os.path.splitext(img)[0]):
          shutil.move(os.path.join(sourceImgPath, scenesFolder, img), os.path.join(destRainSceneFolder, img))

# **Training**

### Configure

In [ ]:
EXP_NAME = "aug2_l1_01_bs_2_epochs_200_400"

### tensorboard

In [ ]:
%tensorboard --logdir runs

### Train

In [ ]:
if not os.path.exists(f"checkpoints/{EXP_NAME}"):
  os.makedirs(f"checkpoints/{EXP_NAME}")
!python -W ignore EECE571T/train.py --dataroot ./datasets/GT-RAIN --name {EXP_NAME} --CUT_mode CUT --lambda_SSIM 0.0 | tee checkpoints/{EXP_NAME}/train.txt

### Save to Drive

In [ ]:
import os
if not os.path.exists('drive/MyDrive/CUT_runs/checkpoints'):
  os.makedirs('drive/MyDrive/CUT_runs/checkpoints')
if not os.path.exists('drive/MyDrive/CUT_runs/runs'):
  os.makedirs('drive/MyDrive/CUT_runs/runs')
!cp -r checkpoints/{EXP_NAME} drive/MyDrive/CUT_runs/checkpoints/
!cp -r runs/{EXP_NAME} drive/MyDrive/CUT_runs/runs/

# **Test**

### Translate & Save

In [ ]:
if not os.path.exists(f"results/{EXP_NAME}"):
  os.makedirs(f"results/{EXP_NAME}")
!python -W ignore EECE571T/test.py --dataroot ./datasets/GT-RAIN --name {EXP_NAME} --CUT_mode CUT --num_test 5000 | tee results/{EXP_NAME}/test.txt

### Compute FID

In [ ]:
!python -W ignore -m pytorch_fid results/{EXP_NAME}/test_latest/images/real_B results/{EXP_NAME}/test_latest/images/fake_B --device cuda:0 | tee results/{EXP_NAME}/fid.txt

### Save to Drive

In [ ]:
if not os.path.exists('drive/MyDrive/CUT_runs/results'):
  os.makedirs('drive/MyDrive/CUT_runs/results')
!cp -r results/{EXP_NAME} drive/MyDrive/CUT_runs/results/

In [ ]:
if not os.path.exists('drive/MyDrive/CUT_runs/results'):
  os.makedirs('drive/MyDrive/CUT_runs/results')
!cp -r results/{EXP_NAME} drive/MyDrive/CUT_runs/results/